## Prepare environment

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
path_code_repo_1 = '<insert here>'

## Load data

In [ ]:
!pip install GitPython

In [ ]:
from langchain.document_loaders import GitLoader # https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/git.html

In [ ]:
loader = GitLoader(repo_path=path_code_repo_1, branch="main", file_filter=lambda file_path: file_path.endswith(".swift") or file_path.endswith(".h") or file_path.endswith(".m"))

In [ ]:
data = loader.load()

In [ ]:
len(data)

## Split text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  texts = text_splitter.split_documents(documents)
  return texts

texts = split_docs(data)
print(len(texts))
print(texts[0].page_content)

## Init ChromaDB

In [ ]:
import os
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(
    openai_api_key=os.environ['OPENAI_API_KEY']
)

In [ ]:
!pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(texts, embeddings)

## Create the Chain

In [ ]:
#from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import VectorDBQA

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo", 
    temperature=0.7, 
    max_tokens=1000
    )

qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=vectordb)

In [ ]:
query = "Can you refactor DependeciesController to use Cocoapods?"
qa.run(query)

In [ ]:
query = "Can you refactor DependenciesController to remove Carthage?"
qa.run(query)

In [ ]:
query = "Show code to refactor DependenciesController to remove Carthage. Output format: markdown"
qa.run(query)